In [1]:
import numpy as np
import scipy
import pandas
import torch
import utils
from scipy import linalg


In [2]:
data_file = '../../../data/Beijing/4x6x12x6/beijing_15k.npy'
data = np.load(data_file, allow_pickle=True).item()
nvec = data['ndims']
data = data['data']

data_dict = {}

fold=0
data_dict['ind'] = torch.tensor(data[fold]['tr_ind'])
data_dict['t'] = data[fold]['tr_T']
data_dict['y'] = data[fold]['tr_y'].astype(np.float32)
data_dict['ind_te'] = data[fold]['te_ind']
data_dict['t_te'] = data[fold]['te_T']
data_dict['y_te'] = data[fold]['te_y'].astype(np.float32)
data_dict['ndims'] = nvec
data_dict['num_node'] = np.sum(nvec)
data_dict['nmod'] = len(nvec)
data_dict['device'] = torch.device("cpu")




In [5]:
L = utils.generate_Lapla(data_dict)



In [27]:
def generate_state_space_Matern_23(data_dict,hyper_dict):
    '''
    For matern 3/2 kernel with given hyper-paras and data,
    generate the parameters of coorspoding state_space_model,
    recall: for each dim of all-node-embedding, the form of state_space_model is iid (independent & identical)
    
    input: data_dict, hyper_dict 
    output: trans mat: F,  stationary covarianc: P_inf

    '''

    D = data_dict['num_node']

    # hyper-para of kernel
    lengthscale = hyper_dict['ls']
    variance = hyper_dict['vars']
    
    lamb = np.sqrt(3)/lengthscale
    
    # F = torch.zeros((2*D, 2*D), device=data_dict['device'])
    F = np.zeros((2*D, 2*D))
    F[:D,:D] = utils.generate_Lapla(data_dict)
    F[:D,D:] = np.eye(D)
    F[D:,:D] = -np.square(lamb) * np.eye(D)
    F[D:,D:] = -2 * lamb *  np.eye(D)

    Q_c = 4 * lamb**3 * variance
    L = np.concatenate([np.zeros(D),np.ones(D)]).reshape(-1,1) 
    Q = - Q_c* np.matmul(L,L.T)

    P_inf = Lyapunov_slover(F,Q)

    return F, P_inf


def Lyapunov_slover(F,Q):
    '''
    For the given mix-process SDE, solve correspoding Lyapunov to get P_{\inf}  
    '''
    
    return linalg.solve_continuous_lyapunov(F, Q)
    

In [59]:
D = 1

# hyper-para of kernel
lengthscale = 1
variance = 1

lamb = np.sqrt(3)/lengthscale

# F = torch.zeros((2*D, 2*D), device=data_dict['device'])
F = np.zeros((2*D, 2*D))
# F[:D,:D] = utils.generate_Lapla(data_dict)

F[:D,D:] = np.eye(D)
F[D:,:D] = -np.square(lamb) * np.eye(D)
F[D:,D:] = -2 * lamb *  np.eye(D)

Q_c = 4 * lamb**3 * variance
L = np.concatenate([np.zeros(D),np.ones(D)]).reshape(-1,1) 
Q = - Q_c* np.matmul(L,L.T)

P_inf = Lyapunov_slover(F,Q)

In [65]:
F_diag =  np.array([[0,1.0,0,0],
                    [-lamb*lamb,-2*lamb,0,0],
                    [0,0,0,1.0],
                    [0,0,-lamb*lamb,-2*lamb]
                    ])
L =  np.array([0,1.0,0,1.0]).reshape(-1,1) 

# F_diag =  np.array([[0,1.0],
#                     [-lamb*lamb,-2*lamb]
#                     ])
# L =  np.array([0,1.0]).reshape(-1,1)


Q_c = 4 * lamb**3 * variance
Q = - Q_c* np.matmul(L,L.T).T
Lyapunov_slover(F_diag,Q)

array([[ 1.00000000e+00, -1.66533454e-16,  1.00000000e+00,
        -1.66533454e-16],
       [ 0.00000000e+00,  3.00000000e+00,  0.00000000e+00,
         3.00000000e+00],
       [ 1.00000000e+00, -1.66533454e-16,  1.00000000e+00,
        -1.66533454e-16],
       [ 0.00000000e+00,  3.00000000e+00,  0.00000000e+00,
         3.00000000e+00]])

In [60]:
P_inf

array([[ 1.00000000e+00, -1.66533454e-16],
       [ 0.00000000e+00,  3.00000000e+00]])

In [44]:
P_inf

array([[ 1.00000000e+00,  1.00000000e+00, -1.66533454e-16,
        -1.66533454e-16],
       [ 1.00000000e+00,  1.00000000e+00, -1.66533454e-16,
        -1.66533454e-16],
       [ 0.00000000e+00,  0.00000000e+00,  3.00000000e+00,
         3.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  3.00000000e+00,
         3.00000000e+00]])

In [16]:
D=2
L = np.concatenate([np.zeros(D),np.ones(D)]).reshape(-1,1) 
np.matmul(L,L.T)

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 1., 1.],
       [0., 0., 1., 1.]])